In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# from subprocess import check_output
# print(check_output(["ls", "data"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
import pandas as pd
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
import re

from keras import backend as K
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Bidirectional, GlobalMaxPool1D
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers.convolutional import Conv1D
from keras.layers import GRU, LSTM
from tensorflow.keras.models import model
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [1]:
def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        elif tag.startswith('R'):
            yield wnl.lemmatize(word, pos='r')
            
        else:
            yield word


def msgProcessing(raw_msg):
    m_w=[]
    words2=[]
    raw_msg=str(raw_msg)
    raw_msg = str(raw_msg.lower())
    #url_stripper= re.sub(r'Email me.*[A-Z]',"",s)
    
    #raw_msg=re.sub(r'\w*[0-9]\w*','', url_stripper)
    raw_msg=re.sub(r'[^a-zA-Z]', ' ', raw_msg)
    
    words=raw_msg.lower().split()
    #Remove words with length lesser than 3 if not w in stops
    for i in words:
        if len(i)>=2:
            words2.append(i)
    stops=set(stopwords.words('english'))
    m_w=" ".join([w for w in words2])
    return(" ".join(lemmatize_all(m_w)))


def helperFunction(df):
    print ("Data Preprocessing!!!")
    cols=['comment_text']
    df=df[cols]
    df.comment_text.replace({r'[^\x00-\x7F]+':''},regex=True,inplace=True)
    num_msg=df[cols].size
    clean_msg=[]
    for i in range(0,num_msg):
        clean_msg.append(msgProcessing(df['comment_text'][i]))
    df['Processed_msg']=clean_msg
    X=df['Processed_msg']
    print ("Data Preprocessing Ends!!!")
    return X


def embedding(train,test):
    tokenizer = Tokenizer(num_words=20000)
    tokenizer.fit_on_texts(train)
    t=len(tokenizer.word_index)+1
    trainsequences = tokenizer.texts_to_sequences(train)
    traindata = pad_sequences(trainsequences, maxlen=100)
    testsequences = tokenizer.texts_to_sequences(test)
    testdata = pad_sequences(testsequences, maxlen=100)
    return traindata, testdata,t



def getTarget(y):
    ytrain=y[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
    return ytrain



df= pd.read_csv("data/train.csv",encoding='latin-1')



X=helperFunction(df)



df2=pd.read_csv("data/test.csv",encoding='latin-1')
df2['comment_text'].fillna('Missing',inplace=True)



X2=helperFunction(df2)



xtrain,xtest,vocab_size=embedding(X,X2)



classes=["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
ytrain=getTarget(df[classes])




def build_CNN_GRU_Model(xtrain,ytrain):
    batch_size=1000
    epochs=5
    model= Sequential()
    model.add(Embedding(20000,32,input_length=100))
    model.add(Conv1D(32,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.3))
    model.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.35))
    model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    model.add(GRU(50,return_sequences=True))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.45))
    model.add(Dense(6,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(xtrain,ytrain,batch_size=batch_size,epochs=epochs)
    model.save("toxic_CNN_GRU.h5")
    pred=model.predict(xtest)
    return pred



pred= build_CNN_GRU_Model(xtrain,ytrain)



def saveCSV(ytest):
    sample_submission = pd.read_csv("data/sample_submission.csv",encoding='latin-1')
    sample_submission[classes] = ytest
    sample_submission.to_csv("toxic.csv", index=False)


saveCSV(pred)

Data Preprocessing!!!


C:\Users\l8527\.conda\envs\tf-image\lib\site-packages\pandas\core\series.py:4479: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-1-7e940b979d7f>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Processed_msg']=clean_msg


Data Preprocessing Ends!!!
Data Preprocessing!!!
Data Preprocessing Ends!!!
Epoch 1/5
160/160 [==============================] - 19s 101ms/step - loss: 0.2736 - accuracy: 0.3943
Epoch 2/5
160/160 [==============================] - 16s 102ms/step - loss: 0.1250 - accuracy: 0.7829
Epoch 3/5
160/160 [==============================] - 16s 103ms/step - loss: 0.1042 - accuracy: 0.9170
Epoch 4/5
160/160 [==============================] - 16s 102ms/step - loss: 0.0822 - accuracy: 0.9730
Epoch 5/5
160/160 [==============================] - 16s 102ms/step - loss: 0.0771 - accuracy: 0.9749


In [2]:
def build_CNN_Model(xtrain,ytrain):
    batch_size=1000
    epochs=5
    model= Sequential()
    model.add(Embedding(20000,32,input_length=100))
    model.add(Conv1D(32,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.3))
    model.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.35))
    model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.45))
    model.add(Dense(6,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(xtrain,ytrain,batch_size=batch_size,epochs=epochs)
    model.save("toxic_CNN.h5")
    pred=model.predict(xtest)
    return pred

pred= build_CNN_Model(xtrain,ytrain)

Epoch 1/5
160/160 [==============================] - 16s 95ms/step - loss: 0.2508 - accuracy: 0.4101
Epoch 2/5
160/160 [==============================] - 15s 96ms/step - loss: 0.1212 - accuracy: 0.7579
Epoch 3/5
160/160 [==============================] - 15s 96ms/step - loss: 0.1120 - accuracy: 0.8825
Epoch 4/5
160/160 [==============================] - 15s 97ms/step - loss: 0.1024 - accuracy: 0.9577
Epoch 5/5
160/160 [==============================] - 16s 102ms/step - loss: 0.0840 - accuracy: 0.9860


In [4]:
def build_CNN_LSTM_Model(xtrain,ytrain):
    batch_size=1000
    epochs=5
    model= Sequential()
    model.add(Embedding(20000,32,input_length=100))
    model.add(Conv1D(32,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.3))
    model.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.35))
    model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    model.add(LSTM(50,return_sequences=True))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.45))
    model.add(Dense(6,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(xtrain,ytrain,batch_size=batch_size,epochs=epochs)
    model.save("toxic_CNN_LSTM.h5")
    pred=model.predict(xtest)
    return pred

pred= build_CNN_LSTM_Model(xtrain,ytrain)

Epoch 1/5
160/160 [==============================] - 21s 114ms/step - loss: 0.2834 - accuracy: 0.4441
Epoch 2/5
160/160 [==============================] - 17s 106ms/step - loss: 0.1288 - accuracy: 0.7651
Epoch 3/5
160/160 [==============================] - 17s 105ms/step - loss: 0.1144 - accuracy: 0.9048
Epoch 4/5
160/160 [==============================] - 17s 105ms/step - loss: 0.1051 - accuracy: 0.9625
Epoch 5/5
160/160 [==============================] - 17s 107ms/step - loss: 0.0922 - accuracy: 0.9798


In [18]:
def build_LSTM_Model(xtrain,ytrain):
    batch_size=1000
    epochs=5
    model= Sequential()
    model.add(Embedding(20000,32,input_length=100))
    model.add(LSTM(50))
    model.add(Dropout(0.25))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.45))
    model.add(Dense(6,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(xtrain,ytrain,batch_size=batch_size,epochs=epochs)
    model.save("toxic_LSTM.h5")
    pred=model.predict(xtest)
    return pred
pred= build_LSTM_Model(xtrain,ytrain)

Epoch 1/5
160/160 [==============================] - 38s 225ms/step - loss: 0.3434 - accuracy: 0.4984
Epoch 2/5
160/160 [==============================] - 37s 233ms/step - loss: 0.1005 - accuracy: 0.8201
Epoch 3/5
160/160 [==============================] - 38s 235ms/step - loss: 0.0573 - accuracy: 0.9431
Epoch 4/5
160/160 [==============================] - 37s 234ms/step - loss: 0.0508 - accuracy: 0.9809
Epoch 5/5
160/160 [==============================] - 37s 234ms/step - loss: 0.0473 - accuracy: 0.9899


In [20]:
def build_GRU_Model(xtrain,ytrain):
    batch_size=1000
    epochs=5
    model= Sequential()
    model.add(Embedding(20000,32,input_length=100))
    model.add(GRU(50))
    model.add(Dropout(0.25))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.45))
    model.add(Dense(6,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(xtrain,ytrain,batch_size=batch_size,epochs=epochs)
    model.save("toxic_GRU.h5")
    pred=model.predict(xtest)
    return pred
pred= build_GRU_Model(xtrain,ytrain)

Epoch 1/5
160/160 [==============================] - 32s 190ms/step - loss: 0.3499 - accuracy: 0.3014
Epoch 2/5
160/160 [==============================] - 31s 191ms/step - loss: 0.0692 - accuracy: 0.8654
Epoch 3/5
160/160 [==============================] - 31s 194ms/step - loss: 0.0543 - accuracy: 0.9405
Epoch 4/5
160/160 [==============================] - 31s 194ms/step - loss: 0.0503 - accuracy: 0.9596
Epoch 5/5
160/160 [==============================] - 31s 191ms/step - loss: 0.0481 - accuracy: 0.9721
